나이브 베이즈(naive bayes)는 확률 기반 머신러닝 분류 알고리즘이다.  
나이브 베이즈 분류 알고리즘은 데이터를 나이브(단순)하고 독립적인 사건으로 가정하고 이 독립 사건들을 베이즈 이론(https://angeloyeo.github.io/2020/01/09/Bayes_rule.html) 에 대입시켜 가장 높은 확률의 레이블로 분류를 실행하는 알고리즘이다.  
***
가우시안 나이브 베이즈(Gaussian Naive Bayes) 분류
특징들의 값들이 정규 분포(가우시안 분포)돼 있다는 가정하에 조건부 확률을 계산하며 연속적인 성질이 있는 특성이 있는 데이터를 분류하는데 적합하다.
***
베르누이 나이브 베이즈(Bernoulli Naive Bayes) 분류
데이터의 특징이 0 또는 1로 표현됐을 때 데이터의 출현 여부에 따라서 0 또는 1로 구분되는 데이터에 사용한다.
***
다항 분포 나이브 베이즈(Multinomial Naive Bayes) 분류
데이터의 특징이 출현 횟수로 표현됐을 때 데이터의 출현 횟수에 따라 값을 달리한 데이터에 사용한다.

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import sklearn

가우시안 나이브 베이즈를 활용한 붓꽃 분류  
iris 데이터를 활용해 데이터의 특징에 따라 붓꽃의 종류를 구분한다.

In [3]:
# 사이킷런에서 제공하는 iris 데이터를 불러오기 우이해서 import 한다.
from sklearn.datasets import load_iris

In [11]:
dataset = load_iris()
# print(type(dataset))
# print(dataset.feature_names)
# 'sepal length (cm)' => 꽃 받침 길이, 'sepal width (cm)' => 꽃 받침 너비
# 'petal length (cm)' => 꽃잎 길이, 'petal width (cm)' => 꽃잎 너비
df = pd.DataFrame(dataset, columns=dataset.feature_names)
# print(type(df))
df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
